# OpenCV - capture Facetime and IP webcams (pictures and streams)
by: frank@aixpact.com  

## Changelog

-


## Setup notebook

In [ ]:
# %matplotlib inline
%matplotlib ipympl
# %matplotlib widget

import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython import display

# from imageio import imread
from matplotlib.image import imread

from mymods.lauthom import *

import sys
sys.path.append('./pycode')

import cv2
from matplotlib.animation import FuncAnimation
import time
from datetime import datetime

Turn on FaceTime camera (green LED on), grab a single image, and then turned it off (LED off), 
saved the image to disk. 
When I have a USB camera plugged in as well, using device 0 or 1 seems to work, and I've captured test images from both.



In [ ]:
url = 'http://192.168.192.42:8080/video'

## Facetime picture

In [ ]:
cap = cv2.VideoCapture(0)

retbool, frame = cap.read()

# Convert to b/w
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Release capture and close camera
cap.release() 
cv2.imwrite('cam_shot_bw.png', gray)
cv2.imwrite('cam_shot_col.png', frame)
cv2.destroyAllWindows()

plt.imshow(imread('cam_shot_col.png'))

## Facetime or webcam stream in seperate window

#### Kill video output - close window

In [ ]:
def kill_stream(cap):
    """Kill the video stream in external video and restart kernel(hard kill of )"""
    import os
    cv2.waitKey(0) # kill camera when a key press is detected
    cap.release() # kill camera
    cv2.waitKey(0) # close window when a key press is detected
    for _ in range(10):
        cv2.destroyAllWindows()
    cv2.waitKey(0) # exit child process(python kernel) - cv2 hangs python on closing windows
    os._exit(00)

#### Open video stream in seperate window

In [ ]:
# http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html#
cap = cv2.VideoCapture(url)

timer = Timer()
while timer.seconds() < 30:
    # Capture frame-by-frame and mirror image
    is_capturing, frame = cap.read()
    mirror = cv2.flip(frame, 1)

    # Covert to grayscale image
    gray = cv2.cvtColor(mirror, cv2.COLOR_BGR2GRAY)

    # Display the camera in a seperate window
    # Stop the camera when typing 'q' in the window
    cv2.imshow('webcam', gray)
    if cv2.waitKey(1) == ord('q') & 0xFF:
        break

# restart kernel when camera hangs
kill_stream(cap)

## Video capture with motion detection (output in jupyter lab)

#### Create Frame class

Keeps track of motion.

In [ ]:
class Frame():
    def __init__(self, source=0):
        self.cap = cv2.VideoCapture(source)
        
        # set frame rate FPS
        self.cap.set(cv2.CAP_PROP_FPS, 40) # 60
        # alternatively use wait for N milisecs to change the frame rate
        cv2.waitKey(1)
        _, frame = self.cap.read()
        mirror = cv2.flip(frame, 1)
        self.bg_gray = cv2.cvtColor(mirror, cv2.COLOR_BGR2GRAY)
        self.bg_col = cv2.cvtColor(mirror, cv2.COLOR_BGR2RGB)
        self.rgb = None
        self.mirror = None
        self.gray = None
        self.blur = None
        self.motions = [(None, None, None, None)]
        self.times = [datetime.now()]
        
    def capture(self):
        is_capturing, frame = self.cap.read()
        self.mirror = cv2.flip(frame, 1)
        self.rgb = cv2.cvtColor(self.mirror, cv2.COLOR_BGR2RGB)
        self.gray = cv2.cvtColor(self.mirror, cv2.COLOR_BGR2GRAY)
        self.blur = cv2.GaussianBlur(self.gray, (21, 21), 0)
        
        diff_frame = cv2.absdiff(self.bg_gray, self.gray)
        thresh_frame = cv2.threshold(diff_frame, 30, 255, cv2.THRESH_BINARY)[1]
        self.thres = cv2.dilate(thresh_frame, None, iterations=2)
        self.get_rectangles()
        
    def get_contours(self):
        """Finding contour of moving object"""
        (_, contours, _) = cv2.findContours(self.thres.copy(), 
                   cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        return contours
        
    def get_rectangles(self):
        for contour in self.get_contours():
            if cv2.contourArea(contour) < 10000:
                continue
            self.times.append(datetime.now())
            (x, y, w, h) = cv2.boundingRect(contour)
            self.motions.append((x, y, w, h))
            # making green rectangle arround the moving object - updating rgb
            self.rgb = cv2.rectangle(self.rgb, (x, y), (x+w, y+h), (0, 255, 0), 3)
        
    def close(self):
        """Stop capturing"""
        self.cap.release()
        

#### Capture frame

In [ ]:
facetime = Frame(0)

In [ ]:
facetime.capture()
plt.imshow(facetime.rgb)
plt.show();

In [ ]:
# Capture new frame
facetime.capture()
plt.imshow(facetime.rgb)
plt.show();

#### Object class paradigm - 2 camera's

In [ ]:
# FuncAnimation needs %matplotlib ipympl

# Initiate the two cameras
# VideoCapture(0) is first connected camera 
# VideoCapture(url) is for ip camera 
cam1 = Frame(0)
cam2 = Frame(0) #url)

# Create two subplots
f = plt.figure(figsize=(12,6))
ax1 = f.add_subplot(121)
ax2 = f.add_subplot(122)
ax1.axis('off')
ax2.axis('off')

# Take with 2 camera's 1 shot and outpot as image
cam1.capture()
cam2.capture()
im1 = ax1.imshow(cam1.rgb)
im2 = ax2.imshow(cam2.blur)

# Interactive animation - update images (new shots)
def update(i):
    cam1.capture()
    cam2.capture()
    im1.set_data(cam1.rgb)
    im2.set_data(cam2.blur)
    timest = cam1.times[-1].strftime("%H:%M:%S %Z")
    plt.suptitle(f'Last move: {timest}')

ani = FuncAnimation(plt.gcf(), update, interval=60)

def close(event):
    if event.key == 'q':
        plt.close(event.canvas.figure)
        cam1.close()
        cam2.close()

cid = plt.gcf().canvas.mpl_connect("key_press_event", close)

plt.show();

#### Functional paradigm

In [ ]:
# FuncAnimation needs %matplotlib ipympl

def mirror_frame(cap):
    """Grap frame and mirror it"""
    is_capturing, frame = cap.read()
    return cv2.flip(frame, 1)

def rgb_frame(cap):
    """Grap frame and mirror it"""
    mirror = mirror_frame(cap)
    return cv2.cvtColor(mirror, cv2.COLOR_BGR2RGB)

def bw_frame(cap):
    """Grap frame and mirror it"""
    mirror = mirror_frame(cap)
    return cv2.cvtColor(mirror, cv2.COLOR_BGR2GRAY)

def blur_frame(cap):
    """Grap frame and mirror it"""
    gray = bw_frame(cap)
    return cv2.GaussianBlur(gray, (21, 21), 0)


# Initiate the two cameras
cap1 = cv2.VideoCapture(0)
cap2 = cv2.VideoCapture(url)

# Create two subplots
f = plt.figure(figsize=(12,6))
ax1 = f.add_subplot(121)
ax2 = f.add_subplot(122)
ax1.axis('off')
ax2.axis('off')

# Take with 2 camera's 1 shot and outpot as image
im1 = ax1.imshow(rgb_frame(cap1))
im2 = ax2.imshow(blur_frame(cap2))

# Interactive animation
def update(i):
    im1.set_data(rgb_frame(cap1))
    im2.set_data(blur_frame(cap2))

ani = FuncAnimation(plt.gcf(), update, interval=100)

def close(event):
    if event.key == 'q':
        plt.close(event.canvas.figure)
        cap1.release()
        cap2.release()

cid = plt.gcf().canvas.mpl_connect("key_press_event", close)

plt.show();

## Motion Detection with 4 camera's

In [ ]:
# Capture frames
facetime = Frame(url)
facetime.capture()
captures = [facetime.rgb, facetime.gray, facetime.blur, facetime.thres]
images = {}

In [ ]:
# 2 images in row
fig12, axes = plt.subplots(2, 2, figsize=(10.5, 5.95))
_ = plt.subplots_adjust(wspace=0, hspace=0) #left=None, bottom=None, right=None, top=None,
_ = plt.suptitle('Motion detection')

# Create subplots
for i, ax in enumerate(axes.flatten()):
    images[i] = ax.imshow(captures[i])
    _ = ax.axis('off')

In [ ]:
# Interactive animation
def update(i):
    """Callback"""
    facetime.capture()
    captures = [facetime.rgb, facetime.gray, facetime.blur, facetime.thres]
    for n, ax in enumerate(axes.flatten()):
        images[n].set_data(captures[n])
        
def close(event):
    if event.key == 'q':
        facetime.close()
        plt.close(event.canvas.figure)
        
ani = FuncAnimation(fig12, update, interval=100) # plt.gcf()

cid = fig12.canvas.mpl_connect("key_press_event", close) # plt.gcf()

plt.show();